In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk import NaiveBayesClassifier
from sklearn.model_selection import train_test_split
from nltk.stem import PorterStemmer
import re

Mounted at /content/drive


In [ ]:
df = pd.read_csv('conteudo')

In [ ]:
df

,texto,polaridade
0,Bem macio e felpudo...recomendo. Preço imbatí...,1.0
1,Produto excepcional! recomendo!!! inovador e ...,1.0
2,recebi o produto antes do prazo mas veio com d...,0.0
3,Bom custo beneficio. Adequado para pessoas que...,1.0
4,"ótmo produto atendeu as expectativas, a entreg...",1.0
...,...,...
116053,"Ótimo jogo, um dos melhores que joguei. Entreg...",1.0
116054,SMARTHPHONE PERFEITO MUITO BOM E A ECONO...,1.0
116055,Eu estou muito chateado com isso gostaria que ...,0.0
116056,O produto anunciado no site dizia ser 100% al...,0.0


Tratamento dos dados, e faz a rede neural.

depois balancear os dados: sugestão, eliminação. Após rede neural

Depois utilizar o naive-bayes

# **Tratamento dos dados**

In [ ]:
nltk.download('stopwords')
stop_portugues = set(stopwords.words('portuguese'))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
for palavra_manter in palavras_a_manter:
    if palavra_manter in stop_portugues:
        stop_portugues.remove(palavra_manter)
        print(f"Palavra '{palavra_manter}' removida do conjunto de stopwords.")
    else:
        print(f"'{palavra_manter}' não encontrada no conjunto de stopwords padrão (já não seria removida).")

Palavra 'não' removida do conjunto de stopwords.
'nao' não encontrada no conjunto de stopwords padrão (já não seria removida).


In [ ]:
for i, palavra in enumerate(sorted(stop_portugues)):
    print(f"{i+1}. {palavra}")

1. a
2. ao
3. aos
4. aquela
5. aquelas
6. aquele
7. aqueles
8. aquilo
9. as
10. até
11. com
12. como
13. da
14. das
15. de
16. dela
17. delas
18. dele
19. deles
20. depois
21. do
22. dos
23. e
24. ela
25. elas
26. ele
27. eles
28. em
29. entre
30. era
31. eram
32. essa
33. essas
34. esse
35. esses
36. esta
37. estamos
38. estar
39. estas
40. estava
41. estavam
42. este
43. esteja
44. estejam
45. estejamos
46. estes
47. esteve
48. estive
49. estivemos
50. estiver
51. estivera
52. estiveram
53. estiverem
54. estivermos
55. estivesse
56. estivessem
57. estivéramos
58. estivéssemos
59. estou
60. está
61. estávamos
62. estão
63. eu
64. foi
65. fomos
66. for
67. fora
68. foram
69. forem
70. formos
71. fosse
72. fossem
73. fui
74. fôramos
75. fôssemos
76. haja
77. hajam
78. hajamos
79. havemos
80. haver
81. hei
82. houve
83. houvemos
84. houver
85. houvera
86. houveram
87. houverei
88. houverem
89. houveremos
90. houveria
91. houveriam
92. houvermos
93. houverá
94. houverão
95. houveríamos
96

In [ ]:
def limpar_texto_simples_portugues(serie_de_dados_texto):
    textos_limpos = []
    for texto_original in serie_de_dados_texto:
        # Garante que a entrada é uma string, trata valores não-string (como NaN)
        if not isinstance(texto_original, str):
            textos_limpos.append("")
            continue

        # 1. Converter para minúsculas
        texto_processado = texto_original.lower()

        # 2. Remover pontuação e números (mantendo apenas letras acentuadas e espaços)
        # Regex: ^a-záàâãéêíóôõúüç\s => aceita letras de 'a' a 'z', acentuadas, 'ç' e espaços
        texto_processado = re.sub(r'[^a-záàâãéêíóôõúüç\s]', ' ', texto_processado)

        # 3. Remover múltiplos espaços em branco e espaços nas bordas
        texto_processado = re.sub(r'\s+', ' ', texto_processado).strip()

        # 4. Tokenização e Remoção de Stopwords
        palavras_filtradas = [
            palavra for palavra in texto_processado.split()
            if palavra not in paradas_portugues and palavra # Garante que a palavra não é vazia
        ]

        textos_limpos.append(' '.join(palavras_filtradas))
    return textos_limpos

In [ ]:
print("\n--- Aplicando a função de limpeza simplificada ao conjunto de dados ---")
df['texto'] = limpar_texto_simples_portugues(df['texto'])


--- Aplicando a função de limpeza simplificada ao conjunto de dados ---


In [ ]:
df

,texto,polaridade
0,bem macio felpudo recomendo preço imbatível en...,1.0
1,produto excepcional recomendo inovador gostoso...,1.0
2,recebi produto antes prazo veio defeito trava ...,0.0
3,bom custo beneficio adequado pessoas fazem uso...,1.0
4,ótmo produto atendeu expectativas entrega ocor...,1.0
...,...,...
116053,ótimo jogo melhores joguei entrega produto loj...,1.0
116054,smarthphone perfeito bom economia energia perf...,1.0
116055,chateado gostaria vcs resolve rápido pq pagand...,0.0
116056,produto anunciado site dizia algodão porém rec...,0.0


In [ ]:
X = df['texto']
y = df['polaridade']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import LabelEncoder

In [ ]:
#Só para conferir o tamanho da base

print(f"Tamanho do conjunto de treino (textos): {len(X_train)}")
print(f"Tamanho do conjunto de teste (textos): {len(X_test)}")
print(f"Tamanho do conjunto de treino (sentimentos): {len(y_train)}")
print(f"Tamanho do conjunto de teste (sentimentos): {len(y_test)}")

Tamanho do conjunto de treino (textos): 81240
Tamanho do conjunto de teste (textos): 34818
Tamanho do conjunto de treino (sentimentos): 81240
Tamanho do conjunto de teste (sentimentos): 34818


In [ ]:
vetorizador = CountVectorizer()
vetorizador.fit(X_train)

X_train = vetorizador.transform(X_train)
X_test = vetorizador.transform(X_test)


# Ler as frases ok?

In [ ]:
dimensao_entrada = len(vetorizador.get_feature_names_out())
print(f"Número de palavras únicas (features) no vocabulário: {dimensao_entrada}")
print(f"Esta será a dimensão da camada de entrada da Rede Neural: {dimensao_entrada}")

Número de palavras únicas (features) no vocabulário: 39547
Esta será a dimensão da camada de entrada da Rede Neural: 39547


In [ ]:
le = LabelEncoder()
le.fit(y)
y_train = le.transform(y_train)
y_test = le.transform(y_test)

# Classificar

In [ ]:
from keras.models import Sequential
from keras.layers import Input, Dense

In [ ]:
model = Sequential([
    Input(shape=(len(vetorizador.get_feature_names_out()),)),
    Dense(100, activation='relu'), # Defini 100 neuronios para as camadas
    Dense(2, activation='sigmoid') # Somente dois neuronios
])
model.compile(loss='sparse_categorical_crossentropy', # perda
              optimizer='adam',
              metrics=['accuracy'])

In [ ]:
history = model.fit(X_train, y_train,
          epochs=2, # epocas aqui é quantas vezes ele se estuda
          verbose=1)  # sempre vai ser o feedback dele

Epoch 1/2
2539/2539 ━━━━━━━━━━━━━━━━━━━━ 133s 52ms/step - accuracy: 0.9097 - loss: 0.2337
Epoch 2/2
2539/2539 ━━━━━━━━━━━━━━━━━━━━ 146s 53ms/step - accuracy: 0.9603 - loss: 0.1095


In [ ]:
scores = model.evaluate(X_test, y_test, verbose=1)
print("Accuracy:", scores[1])

1089/1089 ━━━━━━━━━━━━━━━━━━━━ 15s 14ms/step - accuracy: 0.9387 - loss: 0.1801
Accuracy: 0.9369004368782043


**Vendo os gargalos - desbalanceamento e quantidade de dados**

In [ ]:
df['polaridade'].value_counts()

,count
polaridade,
1.0,80300
0.0,35758


In [ ]:
df['polaridade'].value_counts(normalize=True) * 100

,proportion
polaridade,
1.0,69.189543
0.0,30.810457


esta bastante de proporcional, oq vai inviesar bastante o modelo, assim não vai dar certo

In [ ]:
dados = df.copy()

In [ ]:
# 1. Separar os dados por classe
positivos = dados[dados['polaridade'] == 1]
negativos = dados[dados['polaridade'] == 0]

# 2. Fazer undersampling da classe majoritária (positivos)
positivos_balanceados = positivos.sample(n=len(negativos), random_state=42)

# 3. Juntar as duas classes balanceadas
dados_balanceado = pd.concat([positivos_balanceados, negativos])

# 4. Embaralhar o dataframe final
dados_balanceado = dados_balanceado.sample(frac=1, random_state=42).reset_index(drop=True) # Aleatoriamente

# 5. Verificar proporções
dados_balanceado['polaridade'].value_counts(normalize=True).mul(100).round(2)


,proportion
polaridade,
0.0,50.0
1.0,50.0


In [ ]:
dados = dados_balanceado.copy()

In [ ]:
dados.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 71516 entries, 0 to 71515
Data columns (total 2 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   texto       71516 non-null  object 
 1   polaridade  71516 non-null  float64
dtypes: float64(1), object(1)
memory usage: 1.1+ MB


reduzimos a base para 71516  linhas


In [ ]:
X = dados['texto']
y = dados['polaridade']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

In [ ]:
dimensao_entrada = len(vetorizador.get_feature_names_out())
print(f"Número de palavras únicas (features) no vocabulário: {dimensao_entrada}")
print(f"Esta será a dimensão da camada de entrada da Rede Neural: {dimensao_entrada}")

Número de palavras únicas (features) no vocabulário: 39547
Esta será a dimensão da camada de entrada da Rede Neural: 39547


In [ ]:
#Só para conferir o tamanho da base - agora com menos dados de treino

print(f"Tamanho do conjunto de treino (textos): {len(X_train)}")
print(f"Tamanho do conjunto de teste (textos): {len(X_test)}")
print(f"Tamanho do conjunto de treino (sentimentos): {len(y_train)}")
print(f"Tamanho do conjunto de teste (sentimentos): {len(y_test)}")

Tamanho do conjunto de treino (textos): 50061
Tamanho do conjunto de teste (textos): 21455
Tamanho do conjunto de treino (sentimentos): 50061
Tamanho do conjunto de teste (sentimentos): 21455


In [ ]:
vetorizador = CountVectorizer()
vetorizador.fit(X_train)

X_train = vetorizador.transform(X_train)
X_test = vetorizador.transform(X_test)

In [ ]:
le = LabelEncoder()
le.fit(y)
y_train = le.transform(y_train)
y_test = le.transform(y_test)


In [ ]:
model = Sequential([
    Input(shape=(len(vetorizador.get_feature_names_out()),)),
    Dense(100, activation='relu'), # Defini 100 neuronios para as camadas
    Dense(2, activation='sigmoid') # Somente dois neuronios
])
model.compile(loss='sparse_categorical_crossentropy', # perda
              optimizer='adam',
              metrics=['accuracy'])

In [ ]:
history = model.fit(X_train, y_train,
          epochs=2, # epocas aqui é quantas vezes ele se estuda
          verbose=1)  # sempre vai ser o feedback dele

Epoch 1/2
1565/1565 ━━━━━━━━━━━━━━━━━━━━ 68s 43ms/step - accuracy: 0.9048 - loss: 0.2698
Epoch 2/2
1565/1565 ━━━━━━━━━━━━━━━━━━━━ 67s 43ms/step - accuracy: 0.9614 - loss: 0.1105


In [ ]:
scores = model.evaluate(X_test, y_test, verbose=1)
print("Accuracy:", scores[1])

671/671 ━━━━━━━━━━━━━━━━━━━━ 9s 13ms/step - accuracy: 0.9231 - loss: 0.2167
Accuracy: 0.9263575077056885
